# pivot tables of change accuracy

In [145]:
# imports
import os
import pandas as pd 
import geopandas as gpd 

# paths
folder = r""
cw_path = f"{folder}/CIC/t1-t3_lc_change_values KEY.csv"
points_path = f"{folder}/clean_points/lcc_aa_points_cleaned.gpkg"
fuzzy_path = f"{folder}/clean_points/fuzzy_TA"
write_folder = f"{folder}/clean_points/change_accuracy"
excel_path = f"{write_folder}/REGION_changeAA.xlsx"

In [133]:
# crosswalk
cw = (
    pd.read_csv(cw_path)
    .drop('SMM_added', axis=1)
)

# separate gain and loss
cw[['loss','gain']] = cw['class'].str.split(' to ', n=1, expand=True)
cw.loc[cw['gain'].isna(), 'gain'] = cw.loss

#  aggregate classes
agg_df = (
    pd.DataFrame.from_dict(
        data={
            'Water'                                     : 'Water',
            'Emergent Wetlands'                         : 'Herbaceous',
            'Tree Canopy'                               : 'Tree Canopy',
            'Shrubland'                                 : 'Herbaceous',
            'Low Vegetation'                            : 'Herbaceous',
            'Barren'                                    : 'Barren',
            'Impervious Structures'                     : "Impervious",
            'Other Impervious'                          : "Impervious",
            'Impervious Roads'                          : "Impervious",
            'Tree Canopy Over Impervious Structures'    : 'Tree Canopy',
            'Tree Canopy Over Other Impervious'         : 'Tree Canopy',
            'Tree Canopy Over Impervious Roads'         : 'Tree Canopy',

        },
        orient='index'
    )
    .reset_index()
)
agg_df.columns = ['lc', 'agg_lc']

cw = (
    cw
    .merge(agg_df, left_on='loss', right_on='lc')
    .rename(columns={'agg_lc':"loss_agg"})
    .drop('lc', axis=1)
    .merge(agg_df, left_on='gain', right_on='lc')
    .rename(columns={'agg_lc':"gain_agg"})
    .drop('lc', axis=1)
)

In [183]:
# create single change table
states = ['DC','DE','MD','NY','PA','VA','WV']

# read in points
points = (
    gpd.read_file(points_path, layer='AA_clean')
    .filter(items=['uid', 'state', 'type', 'GrndTruth', 'origMap'], axis=1)
    .rename(columns={'GrndTruth':'GT_ch_pt', 'origMap':'Map_pt'})
    .merge(cw, left_on="GT_ch_pt", right_on='value', how='left')
    .drop(['class', 'value', 'loss','gain'], axis=1)
    .rename(columns={'loss_agg':'GT_T1_agg', 'gain_agg':'GT_T3_agg'})
    .merge(cw, left_on="Map_pt", right_on='value', how='left')
    .drop(['class', 'value', 'loss','gain'], axis=1)
    .rename(columns={'loss_agg':'Map_T1_agg', 'gain_agg':'Map_T3_agg'})
)

In [184]:
# read in fuzzy state data and corrections
fz_list = []
for st in states:
    df = (
        pd.read_csv(f"{fuzzy_path}/{st}/lcc/{st}_lcc_TA_3x3.csv")
    )
    fz_list.append(df.copy())
    df = None

# concat data together
fuz_df = pd.concat(fz_list)
fz_list = None

# iterate corrected TAs and replace info for uids in the corrected data
corrected = [
    f"{fuzzy_path}/correction/lcc_overlap_TA.csv",
    f"{fuzzy_path}/correction/lcc_overlap_TA_dc.csv"
]
for c in corrected:
    #  read in corrections
    tmp = pd.read_csv(c)

    # get list of corrected ids
    uids = tmp.uid.unique().tolist()

    # drop incorrect ids and append corrected
    fuz_df = (
        fuz_df
        .query("uid not in @uids")
    )
    fuz_df = pd.concat([fuz_df, tmp])

    tmp = None

fuz_df = ( 
    fuz_df
    .query("uid >= 0")
    .merge(cw, left_on='lcc', right_on='value', how='left')
    .drop(['class', 'value', 'loss','gain',], axis=1) #  'lcc', 'count'
    .rename(columns={'loss_agg':'Fzy_T1_agg', 'gain_agg':'Fzy_T3_agg'})
) 

In [ ]:
# change or static bsaed on agg classes
points.loc[:, 'Map_agg'] = "Static"
points.loc[points['Map_T1_agg']!=points['Map_T3_agg'], 'Map_agg'] = "Change"
points.loc[:, 'GT_agg'] = "Static"
points.loc[points['GT_T1_agg']!=points['GT_T3_agg'], 'GT_agg'] = "Change"

# change or static based on full schema
points.loc[:, 'Map_all'] = "Static"
points.loc[points['Map_pt']>12, 'Map_all'] = "Change"
points.loc[:, 'GT_all'] = "Static"
points.loc[points['GT_ch_pt']>12, 'GT_all'] = "Change"

# fuzzy - where GT change, did we map change in 3x3 window?
points.loc[:, 'Map_agg_fuzzy'] = points['Map_agg']
uids_ = points.query("Map_agg_fuzzy != GT_agg").uid.unique().tolist()
points = points.set_index("uid")
for uid in uids_:
    #  gt type
    gt = points.loc[uid, 'GT_agg']

    # check if uid has same type 
    if gt == 'Change':
        q = "(uid == @uid) & (Fzy_T1_agg != Fzy_T3_agg)"
        if len(fuz_df.query(q)) > 0:
            points.loc[uid, "Map_agg_fuzzy"] = gt
    else:
        q = "(uid == @uid) & (Fzy_T1_agg == Fzy_T3_agg)" # no data in lcc ta means no change aka static
        if len(fuz_df.query(q)) > 0:
            points.loc[uid, "Map_agg_fuzzy"] = gt

# fuzzy - where GT change, did we map change in 3x3 window?
points.loc[:, 'Map_all_fuzzy'] = points['Map_all']
uids_ = points.query("Map_all_fuzzy != GT_all").index.unique().tolist()
for uid in uids_:
    #  gt type
    gt = points.loc[uid, 'GT_all']

    # check if uid has same type 
    if gt == 'Change':
        q = "(uid == @uid) & (lcc != 65535) & (lcc > 12)"
        if len(fuz_df.query(q)) > 0:
            points.loc[uid, "Map_all_fuzzy"] = gt
    else:
        q = "(uid == @uid) & ((lcc == 0) | (lcc == 65535) | (lcc < 12))" # no data in lcc ta means no change aka static
        if len(fuz_df.query(q)) > 0:
            points.loc[uid, "Map_all_fuzzy"] = gt

points = points.reset_index()

# sum totals and pivot?
agg_tot     = pd.pivot(points.groupby(["Map_agg", "GT_agg"]).size().reset_index(), index='Map_agg', columns='GT_agg', values=0)
all_tot     = pd.pivot(points.groupby(["Map_all", "GT_all"]).size().reset_index(), index='Map_all', columns='GT_all', values=0)
agg_tot_fzy = pd.pivot(points.groupby(["Map_agg_fuzzy", "GT_agg"]).size().reset_index(), index='Map_agg_fuzzy', columns='GT_agg', values=0)
all_tot_fzy = pd.pivot(points.groupby(["Map_all_fuzzy", "GT_all"]).size().reset_index(), index='Map_all_fuzzy', columns='GT_all', values=0)

# add fequencies and UA PA numbers
def _piv_helper(tmp):
    # compute totals
    tmp.loc[:, 'frequency'] = tmp.sum(axis=1)
    tmp.loc['frequency'] = tmp.sum(axis=0)

    # compute PA and UA
    diag_sum = 0
    for lc in ['Change', 'Static']:
        tmp.loc[lc, 'UA'] = tmp.loc[lc, lc] / tmp.loc[lc, 'frequency']
        tmp.loc['PA', lc] = tmp.loc[lc, lc] / tmp.loc['frequency', lc]
        diag_sum += tmp.loc[lc, lc]

    # calculate overall accuracy
    tmp.loc['PA', 'UA'] = diag_sum / tmp.loc['frequency', 'frequency']

    # return
    return tmp

agg_tot = _piv_helper(agg_tot)
all_tot = _piv_helper(all_tot)
agg_tot_fzy = _piv_helper(agg_tot_fzy)
all_tot_fzy = _piv_helper(all_tot_fzy)

if os.path.isfile(excel_path):
    os.remove(excel_path)

all_tot.to_excel(excel_path, sheet_name='change', engine='openpyxl')
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
    agg_tot.to_excel(writer, sheet_name='changeAgg')
    all_tot_fzy.to_excel(writer, sheet_name='changeFuzzy')
    agg_tot_fzy.to_excel(writer, sheet_name='changeAggFuzzy')


In [172]:
def _pivot(points, query_, out_path, fuz_df=None):
    # clean up table
    if os.path.isfile(out_path):
        os.remove(out_path)

    agg_lcs = ['Tree Canopy', 'Impervious', 'Herbaceous', 'Barren', 'Water']
    for agg_lc in agg_lcs:

        # select points where the lc is present in mapped or GT data
        tmp = (
            points
            .query(query_)
            .set_index("uid")
        )

        # assign change types
        tmp.loc[(tmp['GT_T1_agg']==tmp['GT_T3_agg']) & (tmp['GT_T3_agg'] == agg_lc), 'GT_type'] = f'{agg_lc}, No Change' 
        tmp.loc[(tmp['GT_T1_agg']==tmp['GT_T3_agg']) & (tmp['GT_T3_agg'] != agg_lc), 'GT_type'] = f'Not {agg_lc}, No Change' 
        tmp.loc[(tmp['GT_T1_agg']!=tmp['GT_T3_agg']) & ((tmp['GT_T1_agg']!=agg_lc) & (tmp['GT_T3_agg']!=agg_lc)), 'GT_type'] = f'Not {agg_lc}, Change' 
        tmp.loc[(tmp['GT_T1_agg']==agg_lc)&(tmp['GT_type'].isna()), 'GT_type'] = 'Loss'
        tmp.loc[(tmp['GT_T3_agg']==agg_lc)&(tmp['GT_type'].isna()), 'GT_type'] = 'Gain'

        tmp.loc[(tmp['Map_T1_agg']==tmp['Map_T3_agg']) & (tmp['Map_T3_agg'] == agg_lc), 'Map_type'] = f'{agg_lc}, No Change' 
        tmp.loc[(tmp['Map_T1_agg']==tmp['Map_T3_agg']) & (tmp['Map_T3_agg'] != agg_lc), 'Map_type'] = f'Not {agg_lc}, No Change' 
        tmp.loc[(tmp['Map_T1_agg']!=tmp['Map_T3_agg']) & ((tmp['Map_T1_agg']!=agg_lc) & (tmp['Map_T3_agg']!=agg_lc)), 'Map_type'] = f'Not {agg_lc}, Change' 
        tmp.loc[(tmp['Map_T1_agg']==agg_lc)&(tmp['Map_type'].isna()), 'Map_type'] = 'Loss'
        tmp.loc[(tmp['Map_T3_agg']==agg_lc)&(tmp['Map_type'].isna()), 'Map_type'] = 'Gain'

        # IF Fuzzy - do this
        if fuz_df is not None:
            #  iterate where points data disagree, if fuzzy matches GT, update
            uids = tmp.query("GT_type != Map_type").index.unique().tolist()
            for uid in uids:
                # get GT type
                gt_type = tmp.loc[uid, 'GT_type']

                # check if GT type exists in fuzzy window
                # this class should not appear as GT should only be gain or loss
                if gt_type in [f'{agg_lc}, No Change', f'Not {agg_lc}, No Change', f'Not {agg_lc}, Change'] :
                    print(f"Error {uid} - {gt_type}")
                elif gt_type == "Gain":
                    q = "(uid == @uid) & (Fzy_T1_agg != @agg_lc and Fzy_T3_agg == @agg_lc)"
                    if len(fuz_df.query(q)) > 0:
                        tmp.loc[uid, 'Map_type'] = gt_type
                elif gt_type == "Loss":
                    q = "(uid == @uid) & (Fzy_T1_agg == @agg_lc and Fzy_T3_agg != @agg_lc)"
                    if len(fuz_df.query(q)) > 0:
                        tmp.loc[uid, 'Map_type'] = gt_type
                else:
                    print(f"{uid} - expected No Change, Gain, Loss, got {gt_type}")

        # get totals
        tmp = (
            tmp
            .groupby(['GT_type', 'Map_type'])
            .size()
            .reset_index()
        )

        # create pivot table
        tmp = pd.pivot(tmp, index='Map_type', columns='GT_type', values=0)

        # reorganize to be 1-12 and add missing rows/columns 
        types = ['Gain', 'Loss', f'{agg_lc}, No Change', f'Not {agg_lc}, No Change', f'Not {agg_lc}, Change']
        tmp = tmp.reindex(types, axis=0, fill_value=0)
        tmp = tmp.reindex(types, axis=1, fill_value=0)

        # compute totals
        tmp.loc[:, 'frequency'] = tmp.sum(axis=1)
        tmp.loc['frequency'] = tmp.sum(axis=0)

        # write data
        if not os.path.isfile(out_path):
            tmp.to_excel(out_path, sheet_name=agg_lc, engine='openpyxl')
        else:
            with pd.ExcelWriter(out_path, engine='openpyxl', mode='a') as writer:
                tmp.to_excel(writer, sheet_name=agg_lc)

In [173]:
# Where we mapped change, how accurate?
out_path = f"{write_folder}/REGION_changeAA_commission.xlsx"
query_ = "(Map_T1_agg == @agg_lc or Map_T3_agg == @agg_lc) and (Map_T1_agg != Map_T3_agg)"
_pivot(points, query_, out_path, fuz_df=None)

# Where change is ground truthed, what did we miss?
out_path = f"{write_folder}/REGION_changeAA_omission.xlsx"
query_ = "(GT_T1_agg == @agg_lc or GT_T3_agg == @agg_lc) and (GT_T1_agg != GT_T3_agg)"
_pivot(points, query_, out_path, fuz_df=None)

# Where change is ground truthed, what did we miss?
out_path = f"{write_folder}/REGION_changeAA_omission_fuzzy.xlsx"
query_ = "(GT_T1_agg == @agg_lc or GT_T3_agg == @agg_lc) and (GT_T1_agg != GT_T3_agg)"
_pivot(points, query_, out_path, fuz_df=fuz_df)